In [18]:
import numpy as np
import dicom
from tkinter import *
from tkinter import filedialog
from tkinter import Tk

In [19]:
root = Tk()
root.withdraw()
file = filedialog.askopenfilename()
plan = dicom.read_file(file)

In [26]:
from xml.etree.ElementTree import ElementTree
from xml.etree.ElementTree import Element
import xml.etree.ElementTree as etree
from xml.etree.ElementTree import Comment
from xml.dom import minidom
import numpy as np

In [27]:
def GetGantry_and_MLC(plan):
    ##I'm going to start over down here, Mahmoud, and let's summarize what you've been doing so far
    #1. This code here is going to find the cardinal angles that need to be measured.
    #Cardinal angles
    # Gantry angle of start and end cps
    Start = plan.BeamSequence[0].ControlPointSequence[0].GantryAngle
    End = plan.BeamSequence[0].ControlPointSequence[-1].GantryAngle
    #convert to Varian Scale
    if(Start <=180):
        Start= 180 - Start
    else:
        Start = 540-Start
    if(End <=180):
        End = 180 - End
    else:
        End = 540-End
    #print(Start,End)
    possible_cardinals = np.array([Start,0,90,180,270,End])
    #cards= [x in np.arange(Start,End) for x in possible_cardinals]
    #this is only for CC, With CW the line will have reversed comparison operators.
    cards =  possible_cardinals[np.where(np.logical_and(possible_cardinals>=Start, possible_cardinals<=End))]
    print(cards)
    #2. Then find a list of control points that correspond to the surrounding control points at these angles.
    #this time I"ll do the icp calculation inside this loop.
    mlc_new_positions = {} #this is how to create a dictionary
    icp = []
    #looping over the control points to find the cardinal Angles "Varian Coordinates".
    for i in np.arange(0,len(plan.BeamSequence[0].ControlPointSequence)-1):
        current_gantry = plan.BeamSequence[0].ControlPointSequence[i].GantryAngle
        next_gantry = plan.BeamSequence[0].ControlPointSequence[i+1].GantryAngle
        #conversion to Varian Scale.
        current_gantry = 180-current_gantry if current_gantry <= 180 else 540-current_gantry
        next_gantry = 180 -next_gantry if next_gantry <=180 else 540-next_gantry
        for j in cards:
            #print(current_gantry, next_gantry, j)
            #print(np.linspace(current_gantry,next_gantry,np.absolute(current_gantry-next_gantry)+1))
            if j in np.arange(current_gantry,next_gantry+1):
                print ("The cardinal Beam angle in between: ",current_gantry, " and ", next_gantry, " degree 'Varian coordinate'  between control points: ", plan.BeamSequence[0].ControlPointSequence[i].ControlPointIndex, " and ", plan.BeamSequence[0].ControlPointSequence[i+1].ControlPointIndex)
                print("Corresponds to ", str(j))
                #determine MLC positions of current and next control points.
                prev_mlc_pos= plan.BeamSequence[0].ControlPointSequence[i].BeamLimitingDevicePositionSequence[-1].LeafJawPositions
                next_mlc_pos = plan.BeamSequence[0].ControlPointSequence[i+1].BeamLimitingDevicePositionSequence[-1].LeafJawPositions
                #this array is just to calculate the interpolation. 
                temp_mlc = []
                #print(plan.BeamSequence[0].ControlPointSequence[i].BeamLimitingDevicePositionSequence)
                #interpolate to find the exact control point at the cardinal angle
                if j == current_gantry:
                    icp.append(float(plan.BeamSequence[0].ControlPointSequence[i].ControlPointIndex))
                    #print('Current Gantry!')
                    temp_mlc = [float(x) for x in plan.BeamSequence[0].ControlPointSequence[i].BeamLimitingDevicePositionSequence[-1].LeafJawPositions]
                    print('Writing temp MLC for Current Gantry.')
                    #write to the dictionary the temp_mlc with the gantry angle in Varian Standard as the key.
                    mlc_new_positions[j]=temp_mlc
                    #print (plan.BeamSequence[0].ControlPointSequence[i].ControlPointIndex)
                elif j == next_gantry:#ahhh you got me on this one was supposed to be an else if
                    icp.append(float(plan.BeamSequence[0].ControlPointSequence[i+1].ControlPointIndex))
                    #print (plan.BeamSequence[0].ControlPointSequence[i+1].ControlPointIndex)
                    temp_mlc = [float(x) for x in plan.BeamSequence[0].ControlPointSequence[i+1].BeamLimitingDevicePositionSequence[-1].LeafJawPositions]
                    print('Next Gantry!')
                    mlc_new_positions[j] = temp_mlc
                else:
                    print('Interp gantry')
                    icp_temp = plan.BeamSequence[0].ControlPointSequence[i].ControlPointIndex*(1-((j-current_gantry)/(next_gantry-current_gantry)))+plan.BeamSequence[0].ControlPointSequence[i+1].ControlPointIndex*(1-((next_gantry-j)/(next_gantry-current_gantry)))
                    cp_index = float(plan.BeamSequence[0].ControlPointSequence[i].ControlPointIndex)
                    cp_index_next = float(plan.BeamSequence[0].ControlPointSequence[i+1].ControlPointIndex)
                    icp.append(icp_temp)
                    #loop through all MLC positions
                    print('checking mlc positions')
                    print('prev, next, interp')#uncomment the printed line below to actually get these values.
                    for x in np.arange(0,len(prev_mlc_pos)):
                        #y = y_0 + (x-x_0)*((y_1-y_0)/(x_1-x_0))
                        temp_interp = float(prev_mlc_pos[x])+(icp_temp-cp_index)*((float(next_mlc_pos[x])-float(prev_mlc_pos[x]))/(cp_index_next-cp_index))
                        temp_mlc.append(temp_interp)
                        #print(float(prev_mlc_pos[x]),next_mlc_pos[x], temp_interp)
                    mlc_new_positions[j] = temp_mlc
                print (mlc_new_positions[j])
                print('--------------------------')
    print('cardinal angles are: ', cards)
    print('control points are: ',icp)
    #print(mlc_new_positions)
    print ('Done!')
    return mlc_new_positions

In [28]:
MLC_Pos = GetGantry_and_MLC(plan)
print(type(MLC_Pos))

[   1.   90.  180.  270.  359.]
The cardinal Beam angle in between:  1.0  and  3.0  degree 'Varian coordinate'  between control points:  0  and  1
Corresponds to  1.0
Writing temp MLC for Current Gantry.
[-0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, 7.91, 5.35, 4.71, 3.97, 4.96, 5.65, 8.95, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, 19.55, 22.25, 22.55, 22.85, 22.55, 21.95, 18.55, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.2

In [29]:
for gantry,mlc_pos in MLC_Pos.items():
    #this has your gantry angles and your control points.
    print(gantry,mlc_pos)

1.0 [-0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, 7.91, 5.35, 4.71, 3.97, 4.96, 5.65, 8.95, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, 19.55, 22.25, 22.55, 22.85, 22.55, 21.95, 18.55, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25]
90.0 [-0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -5.125, -

In [30]:
list(MLC_Pos)[0]
len(MLC_Pos[list(MLC_Pos)[0]][60:])
list(MLC_Pos)[1:]
len(MLC_Pos)

5

In [31]:
#build the XML file
#first the header.
'''
<VarianResearchBeam SchemaVersion="1.0">
<!--Generated from python-->
    <SetBeam>
        <Id>1234</Id>
        <MLCModel>NDS120HD</MLCModel>
        <Accs>
            <Acc2>3317</Acc2>
        </Accs>
        <ControlPoints>
            <Cp>
                <SubBeam>
                    <Seq>0</Seq>
                    <Name>MV Outside</Name>
                </SubBeam>
                <Energy>6x</Energy>
                <Mu>0</Mu>
                <DRate>400</DRate>
                <GantryRtn>180.0</GantryRtn>
                <CollRtn>180</CollRtn>
                <Y1>2.5</Y1>
                <Y2>2.5</Y2>
                <X1>2.5</X1>
                <X2>2.5</X2>
            </Cp>
            <!--Now loop through the rest of the control points-->
     </SetBeam>
</VarianResearchBeam>
'''

'\n<VarianResearchBeam SchemaVersion="1.0">\n<!--Generated from python-->\n    <SetBeam>\n        <Id>1234</Id>\n        <MLCModel>NDS120HD</MLCModel>\n        <Accs>\n            <Acc2>3317</Acc2>\n        </Accs>\n        <ControlPoints>\n            <Cp>\n                <SubBeam>\n                    <Seq>0</Seq>\n                    <Name>MV Outside</Name>\n                </SubBeam>\n                <Energy>6x</Energy>\n                <Mu>0</Mu>\n                <DRate>400</DRate>\n                <GantryRtn>180.0</GantryRtn>\n                <CollRtn>180</CollRtn>\n                <Y1>2.5</Y1>\n                <Y2>2.5</Y2>\n                <X1>2.5</X1>\n                <X2>2.5</X2>\n            </Cp>\n            <!--Now loop through the rest of the control points-->\n     </SetBeam>\n</VarianResearchBeam>\n'

In [52]:
root = Element ("VarianResearchBeam")
tree= ElementTree(root)
root.set('SchemaVersion','1.0')
comment = Comment('Build from python')
root.append(comment)
setBeam = Element('SetBeam')
root.append(setBeam)
#inner tags
id_tag = Element('Id')
id_tag.text='1234'
setBeam.append(id_tag)
mlcModel = Element('MLCModel')
mlcModel.text = 'NDS120HD'
setBeam.append(mlcModel)
Accs = Element('Accs')
Acc2 = Element('Acc2')
Acc2.text = '3317'
Accs.append(Acc2)
setBeam.append(Accs)
ControlPoints = Element('ControlPoints')
cp = Element('Cp')
SubBeam = Element('SubBeam')
Sequence = Element('Sequence')
Sequence.text = '0'
Name = Element('Name')
Name.text = 'MV Outside'
SubBeam.append(Sequence)
SubBeam.append(Name)
cp.append(SubBeam)
Energy = Element('Energy')
Energy.text = '6x'
cp.append(Energy)
MU=  Element('MU')
MU.text = '0'
cp.append(MU)
drate = Element('DRate')
drate.text = '400'
cp.append(drate)
Gantryrtn = Element('GantryRtn')
# gantry Angle
Gantryrtn.text = list(MLC_Pos)[0]
cp.append(Gantryrtn)
collrtn = Element('CollRtn')
collrtn.text = '180'
cp.append(collrtn)
#jaw size
y1 = Element('Y1')
y2 = Element('Y2')
x1 = Element('X1')
x2 = Element('X2')
y1.text = y2.text = x1.text = x2.text = '2.5'
cp.append(y1)
cp.append(y2)
cp.append(x1)
cp.append(x2)
mlc = Element('Mlc')
mlc_id = Element('ID')
mlc_id.text  = '1'
mlc.append(mlc_id)
mlc_b = Element('B')
j = ','
# MLC B
mlc_b.text = MLC_Pos[list(MLC_Pos)[0]][60:]
mlc.append(mlc_b)
mlc_a = Element('A')
# MLC A
mlc_a.text = MLC_Pos[list(MLC_Pos)[0]][:60]
mlc.append(mlc_a)
cp.append(mlc)

ControlPoints.append(cp)


#now loop through the other control poitns.
for gantry in list(MLC_Pos)[1:]:
    #create blank control poitn with no change in parameters.
    cp_nc = Element('Cp')
    GANTRY = Element ('GantryRtn')
    GANTRY.text= gantry 
    cp_nc.append(GANTRY)
    mlc = Element('Mlc')
    mlc_id = Element('ID')
    mlc_id.text  = '1'
    mlc.append(mlc_id)
    mlc_b = Element('B')
    j = ','
    # MLC B
    mlc_b.text = MLC_Pos[gantry][60:]
    mlc.append(mlc_b)
    mlc_a = Element('A')
    # MLC A
    mlc_a.text = MLC_Pos[gantry][:60]
    mlc.append(mlc_a)
    cp.append(mlc)
dummy_cp = element ('Cp')
ControlPoints.append(dummy_cp)
ControlPoints.append(cp_nc)

    
    #________________________________________________
    # I M A G I N G  S E C T I O N 
    #________________________________________________
    
    #create imaging points with odd control point number.
    Imaging_Parameters = Element ('ImagingParameters')
    Outside_Treatment = Element ('OutsideTreatment')
    Max_MU = Element ('MaxMu')
    Max_MU.text = '100'
    Outside_Treatment.append(Max_MU)
    Imaging_points = Element ('ImagingPoints')

for i in np.arange(0,len(MLC_Pos)):  
    Imaging_Point = Element ('imagingPoint')
    Img_cp = Element('Cp')
    Img_cp.text = 2*i+1
    Imaging_Point.append(Img_cp)
    Acquisition = Element ('Acquisition')
    Acquisition_Id = Element ('AcquisitionId')
    Acquisition_Id.text = '0'
    Acquisition_Specs = Element ('AcquisitionSpecs')
    Acquisition_Parameters = Element ('AcquisitionParameters')
    Image_Mode = Element ('ImgMode')
    Image_Mode.set('id','Highres')
    Mv = Element ('MV')
    Calibration_Set = Element ('CalibrationSet')
    Calibration_Set.text = 'DefaultCalibrationSetId'
    Acquisition_Parameters.append(Image_Mode)
    Acquisition_Parameters.append(Calibration_Set)
    Acquisition_Parameters.append(Mv)
    Acquisition.append(Acquisition_Id)
    Acquisition_Specs.append(Acquisition_Specs)
    Acquisition.append(Acquisition_Parameters)
    Mv_d = Element ('Mvd')
    positions = Element ('Positions')
    Lateral = Element ('Lat')
    Lateral.text = '0'
    Longitudinal = Element ('Lng')
    Longitudinal.text='0'
    Vertical = Element ('vrt')
    Vertical.text = '-180'
    pitch = Element ('Pitch')
    pitch.text='0'
    positions.append(Lateral)
    positions.append(Longitudinal)
    positions.append(Vertical)
    positions.append(pitch)
    Mv_d.append(positions)
    Imaging_Point.append(Acquisition)
    Imaging_Point.append(Mv_d)
    setBeam.append(ControlPoints)
Imaging_Parameters.append(Imaging_points)
setBeam.append(Imaging_Parameters)
root.append(setBeam)

In [53]:
with open('testXML','wb') as xmlfile:
    rough_string = etree.tostring(root)
    parsed = minidom.parseString(rough_string)
    pretty_tree = parsed.toprettyxml(indent='    ')
    #pretty_tree.write(xmlfile)
    print(pretty_tree)

TypeError: cannot serialize 1.0 (type float64)